In [39]:
import pandas as pd
# 新可转债上市预测
file = 'data/机器学习新债数据.xlsx'
data = pd.read_excel(file,index_col=0)

In [10]:
data.head()

,bondPremRatio,call_trigger_date,call_trigger_rate,closePriceBond,code,coupon,firstDate,industry,neg_market_value,put_trigger_date,put_trigger_rate,rate,remainSize,secShortNameBond
0,2.1033,30,130,104.180,110041,0.4,20180109,公用事业,1.748131e+10,30.0,70.0,AAA,18.7522,蒙电转债
1,12.0147,30,130,112.379,128024,0.2,20180112,银行,8.327123e+10,-1.0,-1.0,AAA,100.0000,宁行转债
2,7.9547,30,130,101.760,110042,0.2,20180115,国防军工,2.369592e+10,30.0,70.0,AAA,24.0000,航电转债
3,-0.5855,30,130,98.080,113017,0.3,20180115,传媒,9.145756e+09,30.0,70.0,AA+,15.6000,吉视转债
4,2.8069,30,130,124.530,128029,0.3,20180116,轻工制造,2.712261e+10,30.0,70.0,AA+,12.0000,太阳转债


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458 entries, 0 to 457
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bondPremRatio      458 non-null    float64
 1   call_trigger_date  458 non-null    int64  
 2   call_trigger_rate  458 non-null    int64  
 3   closePriceBond     458 non-null    float64
 4   code               458 non-null    int64  
 5   coupon             458 non-null    float64
 6   firstDate          458 non-null    int64  
 7   industry           458 non-null    object 
 8   neg_market_value   458 non-null    float64
 9   put_trigger_date   454 non-null    float64
 10  put_trigger_rate   454 non-null    float64
 11  rate               458 non-null    object 
 12  remainSize         458 non-null    float64
 13  secShortNameBond   458 non-null    object 
dtypes: float64(7), int64(4), object(3)
memory usage: 53.7+ KB


In [40]:
def convert_permratio_inner(bondPremRatio):
    return 100/(bondPremRatio/100+1)

# 队列执行map操作
data['innerValue'] = data['bondPremRatio'].map(lambda x:convert_permratio_inner(x))

In [41]:
rate_map = {
    'A':0,
    'A+':1,
    'AA-':2,
    'AA':3,
    'AA+':4,
    'AAA':5
}

data['rate'] = data['rate'].map(lambda x:rate_map.get(x,-1))

In [42]:
industry_map = {
'化工':0,
'医药生物':    1,
'机械设备':    2,
'电子':      3,
'电气设备':   4,
'轻工制造':    5,
'汽车':      6,
'有色金属':   7,
'计算机':     8,
'建筑装饰':    9,
'公用事业':    10,
'农林牧渔':    11,
'银行':    12,
'纺织服装':    13,
'食品饮料':    14,
'国防军工':     15,
'通信':       16,
'非银金融':     17,
'交通运输':     18,
'家用电器':     19,
'建筑材料':     20,
'采掘':       21,
'钢铁':       22,
'传媒':       23,
'商业贸易':    24,
'休闲服务':  25,
}
data['industry']=data['industry'].map(lambda x:industry_map.get(x,-1))

In [43]:
data.head()

,bondPremRatio,call_trigger_date,call_trigger_rate,closePriceBond,code,coupon,firstDate,industry,neg_market_value,put_trigger_date,put_trigger_rate,rate,remainSize,secShortNameBond,innerValue
0,2.1033,30,130,104.180,110041,0.4,20180109,10,1.748131e+10,30.0,70.0,5,18.7522,蒙电转债,97.940027
1,12.0147,30,130,112.379,128024,0.2,20180112,12,8.327123e+10,-1.0,-1.0,5,100.0000,宁行转债,89.273997
2,7.9547,30,130,101.760,110042,0.2,20180115,15,2.369592e+10,30.0,70.0,5,24.0000,航电转债,92.631446
3,-0.5855,30,130,98.080,113017,0.3,20180115,23,9.145756e+09,30.0,70.0,4,15.6000,吉视转债,100.588948
4,2.8069,30,130,124.530,128029,0.3,20180116,5,2.712261e+10,30.0,70.0,4,12.0000,太阳转债,97.269736


In [83]:
data = data[data.isna().any(axis=1)==False]
target = data['closePriceBond']

train_data = data.copy()
train_data.drop('secShortNameBond',inplace=True,axis=1)
# 溢价率(反应在价值中了)
train_data.drop('bondPremRatio',inplace=True,axis=1)
# 首日价格,要计算的部分
train_data.drop('closePriceBond',axis=1,inplace=True)
train_data.drop('firstDate',axis=1,inplace=True)
train_data.drop('code',axis=1,inplace=True)

In [84]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_data,target,random_state=4)

In [50]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10],
    'max_features': [4,5,6],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [30]
}
# Create a based model
rf = RandomForestRegressor()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [85]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [5, 10],
                         'max_features': [4, 5, 6],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [30]},
             verbose=2)

In [86]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 5,
 'max_features': 6,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 30}

In [87]:
rf_ = RandomForestRegressor(**grid_search.best_params_)
rf_.fit(X_train,y_train)
y_pred_grid = rf_.predict(X_test)

In [88]:
def evaluate_(y_test,y_test_pre):
    import sklearn.metrics as sm
    import numpy as np
    print('mean absolute error=',round(sm.mean_absolute_error(y_test,y_test_pre),2))
    print('mean squared error=',round(sm.mean_squared_error(y_test,y_test_pre),2))
    print('median absolute error=',round(sm.median_absolute_error(y_test,y_test_pre),2))
    print('explained variance score=',round(sm.explained_variance_score(y_test,y_test_pre),2))
    print('R2 score=',round(sm.r2_score(y_test,y_test_pre),2))

#     mape = 100 * np.mean(errors / test_labels)
#     accuracy = 100 - mape
#     print('Model Performance')
#     print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
#     print('Accuracy = {:0.2f}%.'.format(accuracy))

In [89]:
# rf_.score(X_test, y_test)
evaluate_(y_test,y_pred_grid)

mean absolute error= 9.29
mean squared error= 129.3
median absolute error= 8.0
explained variance score= 0.18
R2 score= 0.17


In [90]:
import numpy as np
importances = rf_.feature_importances_
importances=list(map(lambda x:round(x*100,0),importances))
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print("%2d) %-*s %.2f" % (f + 1, 30, X_train.columns[indices[f]], importances[indices[f]]))

 1) neg_market_value               43.00
 2) innerValue                     26.00
 3) remainSize                     13.00
 4) industry                       7.00
 5) rate                           6.00
 6) coupon                         3.00
 7) put_trigger_rate               1.00
 8) put_trigger_date               1.00
 9) call_trigger_rate              0.00
10) call_trigger_date              0.00


In [96]:
import joblib
joblib.dump(rf_, "train_model.m")
# clf = joblib.load("train_model.m")
# clf.predict(x)

['train_model.m']

In [95]:
with open('new_bond_tree.dot','w') as f:
    tree.export_graphviz(rf_.estimators_[0],feature_names=list(train_data.columns),out_file=f)

In [92]:
from sklearn import tree,preprocessing

In [99]:
input_data = {
'innerValue':convert_permratio_inner((30.980/28.96-1)*100),
'call_trigger_date':30,
'call_trigger_rate':130, # 强赎
'coupon':0.1,
'industry':industry_map.get('电气设备'),
# 'neg_market_value':21042642880,
'put_trigger_date':30,
'put_trigger_rate':70, # triggerRate 回售触发票面比例（%）
'rate':rate_map.get('A+'),
'remainSize':4.3*10**8,
 'neg_market_value':52.13*10**8
}

In [100]:
df_real = pd.DataFrame([input_data])
rf_.predict(df_real)[0]

112.86453530357443